In [1]:
import torch
from PIL import Image
from torchvision import transforms
from clip import clip
from torchvision.datasets import CIFAR10

/storage/users/rkashefi/miniconda3/envs/ad_cl/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/storage/users/rkashefi/miniconda3/envs/ad_cl/lib/python3.9/site-packages/torch/cuda/__init__.py:141: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11070). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


In [5]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [6]:
model, transform = clip.load("ViT-B/32", device=device)

In [7]:
cifar10_dataset = CIFAR10(root='./data', train=True, download=True, transform=transform)

Files already downloaded and verified


# One image

In [29]:
image, _ = cifar10_dataset[0]
image = image.unsqueeze(0).to(device)
image_features1 = model.encode_image(image)

rotated_image = transforms.functional.to_pil_image(image.cpu().squeeze(0))
rotated_image = transforms.functional.rotate(rotated_image, 90)
rotated_image = transform(rotated_image).unsqueeze(0).to(device)
image_features2 = model.encode_image(rotated_image)

diff = torch.pow((image_features1 - image_features2), 2)

# One image + text

In [24]:
text = 'car'
image, _ = cifar10_dataset[0]
image = image.unsqueeze(0).to(device)
image_features1 = model.encode_image(image)
tokenized_text = clip.tokenize([text]).to(device)
text_features1 = model.encode_text(tokenized_text)
first_embedding = torch.matmul(image_features1. T, text_features1)
first_embedding = torch.diagonal(first_embedding)

text2 = 'rotate 90 car'
rotated_image = transforms.functional.to_pil_image(image.cpu().squeeze(0))
rotated_image = transforms.functional.rotate(rotated_image, 90)
rotated_image = transform(rotated_image).unsqueeze(0).to(device)
image_features2 = model.encode_image(rotated_image)
tokenized_text2 = clip.tokenize([text2]).to(device)
text_features2 = model.encode_text(tokenized_text2)
second_embedding = torch.matmul(image_features2.T, text_features2)
second_embedding = torch.diagonal(second_embedding)

diff = torch.pow((first_embedding - second_embedding), 2)


# One text

In [30]:
text = 'car'
tokenized_text = clip.tokenize([text]).to(device)
first_embedding = model.encode_text(tokenized_text)

text2 = 'rotate 90 car'
tokenized_text2 = clip.tokenize([text2]).to(device)
second_embedding = model.encode_text(tokenized_text2)

diff = torch.pow((first_embedding - second_embedding), 2)

# All text -> clip_embedding3.py

# All images + Texts -> clip_embedding2.py
# All images -> clip_embeddig.py

# Softmax (All texts)

In [7]:
import pickle
import os
import torch
import pandas as pd

In [8]:
augs = os.listdir('./outputs')
for aug in augs:
    with open(f'./outputs/{aug}', 'r') as file:
        lines = file.readlines()
        lines.pop(0)
    vals = []
    for line in lines:
        vals.append(float(line.split(':')[-1].split('\n')[0]))
    
    vals = torch.tensor(vals)
    vals_softmax = torch.nn.functional.softmax(vals).detach().cpu().numpy()
    with open(f'./softmax3/{aug}', 'w') as file:
        for val in vals_softmax:
            file.write(str(val)+'\n')

/tmp/ipykernel_239641/2651461328.py:11: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  vals_softmax = torch.nn.functional.softmax(vals).detach().cpu().numpy()


In [10]:
augs = os.listdir('./softmax3')
augs_probs = {}
for i in range(10):
    augs_probs[i] = {}

for aug in augs:
    with open(f'./softmax3/{aug}', 'r') as file:
        lines = file.readlines()

    for i, line in enumerate(lines):
        # augs_probs[i].append({
        #     aug.split('.')[0] : float(line.split('\n')[0])
        # })
        augs_probs[i][aug.split('.')[0]] = float(line.split('\n')[0])

In [11]:
augs_probs[0]

{'random_crop': 0.89308196,
 'rot90': 5.50984e-07,
 'gaussian_blur': 4.9482118e-05,
 'gaussian_noise': 0.05817404,
 'motion_blur': 0.00029707546,
 'jpeg_compression': 0.94662815,
 'shot_noise': 4.5826877e-05,
 'flip': 0.5687391,
 'brightness': 0.0076125097,
 'snow': 0.23965962,
 'fog': 0.17946286,
 'glass_blur': 0.00046986487,
 'elastic_transform': 0.04725114,
 'contrast': 0.92349863,
 'speckle_noise': 1.9689242e-05,
 'color_jitter': 0.0920094,
 'saturate': 7.1688926e-05,
 'zoom_blur': 5.5773227e-05,
 'spatter': 0.03198661,
 'rot270': 1.4107583e-06,
 'pixelate': 0.07810214,
 'impulse_noise': 8.289694e-05,
 'defocus_blur': 3.927535e-05}

In [12]:
augs_sorted = {}
for key in augs_probs.keys():
    augs_sorted[key] = {k: v for k, v in sorted(augs_probs[key].items(), key=lambda item: item[1])}

In [13]:
import pickle
with open('./softmax_sorted3.pkl', 'wb') as file:
    pickle.dump(augs_sorted, file)

In [14]:
import pickle
with open('./softmax_sorted3.pkl', 'rb') as file:
    oo = pickle.load(file)

In [15]:
for key in augs_sorted.keys():
    for key2 in augs_sorted[key].keys():
        print(key2, augs_sorted[key][key2])

rot90 5.50984e-07
rot270 1.4107583e-06
speckle_noise 1.9689242e-05
defocus_blur 3.927535e-05
shot_noise 4.5826877e-05
gaussian_blur 4.9482118e-05
zoom_blur 5.5773227e-05
saturate 7.1688926e-05
impulse_noise 8.289694e-05
motion_blur 0.00029707546
glass_blur 0.00046986487
brightness 0.0076125097
spatter 0.03198661
elastic_transform 0.04725114
gaussian_noise 0.05817404
pixelate 0.07810214
color_jitter 0.0920094
fog 0.17946286
snow 0.23965962
flip 0.5687391
random_crop 0.89308196
contrast 0.92349863
jpeg_compression 0.94662815
rot90 2.7769297e-06
spatter 4.25254e-06
rot270 5.629429e-06
speckle_noise 6.388021e-05
shot_noise 0.00016737344
gaussian_blur 0.00021193536
fog 0.00024298571
gaussian_noise 0.0003778701
random_crop 0.000566664
impulse_noise 0.00096235395
saturate 0.0010788814
defocus_blur 0.0012034562
color_jitter 0.0016476682
contrast 0.0026139228
pixelate 0.002710357
elastic_transform 0.023520842
jpeg_compression 0.02550133
motion_blur 0.030642994
zoom_blur 0.0463677
glass_blur 0.1

In [16]:
import csv

with open('softmax_sorted3.csv', 'w', newline='') as csvfile:
    filewriter = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
    for key in augs_sorted.keys():
        filewriter.writerow([str(key)])
        for key2 in augs_sorted[key].keys():
            # print(key2, augs_sorted[key][key2])
            filewriter.writerow([str(key2), str(augs_sorted[key][key2])])

In [17]:
import pickle
with open('/storage/users/rkashefi/rojina/new_contrastive/clip_vec/softmax_sorted3.pkl', 'rb') as file:
    loaded_data = pickle.load(file)

rearranged_data = {key: {inner_key: value for inner_key, value in inner_dict.items()} for key, inner_dict in loaded_data.items()}
df = pd.DataFrame.from_dict(rearranged_data, orient='index')
df = df.transpose()
df.columns = [
    "Airplane",
    "Automobile",
    "Bird",
    "Cat",
    "Deer",
    "Dog",
    "Frog",
    "Horse",
    "Ship",
    "Truck"
]
df.to_csv('result3.csv')

# Softmax (All images + texts)

In [33]:
import pickle
import os
import torch
import pandas as pd

In [4]:
augs = os.listdir('./outputs')
for aug in augs:
    with open(f'./outputs/{aug}', 'r') as file:
        lines = file.readlines()
        lines.pop(0)
    vals = []
    for line in lines:
        vals.append(float(line.split(':')[-1].split('\n')[0]))
    
    vals = torch.tensor(vals)
    vals_softmax = torch.nn.functional.softmax(vals).detach().cpu().numpy()
    with open(f'./softmax2/{aug}', 'w') as file:
        for val in vals_softmax:
            file.write(str(val)+'\n')

/tmp/ipykernel_194511/1152860754.py:11: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  vals_softmax = torch.nn.functional.softmax(vals).detach().cpu().numpy()


In [5]:
augs = os.listdir('./softmax2')
augs_probs = {}
for i in range(10):
    augs_probs[i] = {}

for aug in augs:
    with open(f'./softmax2/{aug}', 'r') as file:
        lines = file.readlines()

    for i, line in enumerate(lines):
        # augs_probs[i].append({
        #     aug.split('.')[0] : float(line.split('\n')[0])
        # })
        augs_probs[i][aug.split('.')[0]] = float(line.split('\n')[0])

In [6]:
augs_probs[0]

{'random_crop': 1.4700148e-05,
 'rot90': 4.5452645e-11,
 'gaussian_blur': 6.046373e-07,
 'gaussian_noise': 0.035545077,
 'motion_blur': 1.682911e-08,
 'jpeg_compression': 0.50507843,
 'shot_noise': 6.216313e-05,
 'flip': 0.20082706,
 'brightness': 0.0006293851,
 'snow': 0.011818655,
 'fog': 3.7417399e-06,
 'glass_blur': 5.34637e-06,
 'elastic_transform': 0.022836406,
 'contrast': 0.0014445763,
 'speckle_noise': 0.00020597634,
 'color_jitter': 3.89396e-06,
 'saturate': 5.009241e-05,
 'zoom_blur': 9.100349e-05,
 'spatter': 0.061149087,
 'rot270': 1.000543e-11,
 'pixelate': 9.606571e-06,
 'impulse_noise': 0.00024814048,
 'defocus_blur': 3.8540216e-11}

In [7]:
augs_sorted = {}
for key in augs_probs.keys():
    augs_sorted[key] = {k: v for k, v in sorted(augs_probs[key].items(), key=lambda item: item[1])}

In [8]:
import pickle
with open('./softmax_sorted2.pkl', 'wb') as file:
    pickle.dump(augs_sorted, file)

In [9]:
import pickle
with open('./softmax_sorted2.pkl', 'rb') as file:
    oo = pickle.load(file)

In [10]:
for key in augs_sorted.keys():
    for key2 in augs_sorted[key].keys():
        print(key2, augs_sorted[key][key2])

rot270 1.000543e-11
defocus_blur 3.8540216e-11
rot90 4.5452645e-11
motion_blur 1.682911e-08
gaussian_blur 6.046373e-07
fog 3.7417399e-06
color_jitter 3.89396e-06
glass_blur 5.34637e-06
pixelate 9.606571e-06
random_crop 1.4700148e-05
saturate 5.009241e-05
shot_noise 6.216313e-05
zoom_blur 9.100349e-05
speckle_noise 0.00020597634
impulse_noise 0.00024814048
brightness 0.0006293851
contrast 0.0014445763
snow 0.011818655
elastic_transform 0.022836406
gaussian_noise 0.035545077
spatter 0.061149087
flip 0.20082706
jpeg_compression 0.50507843
rot270 6.5782834e-13
rot90 9.095484e-13
fog 2.3932552e-11
random_crop 2.948997e-11
defocus_blur 1.28756135e-08
glass_blur 1.1554168e-06
gaussian_blur 1.3998036e-06
motion_blur 1.897267e-06
color_jitter 8.565002e-06
shot_noise 8.6292575e-06
contrast 3.3346434e-05
impulse_noise 0.00014272494
speckle_noise 0.0002912671
saturate 0.0003100435
zoom_blur 0.0005096204
gaussian_noise 0.00064237794
snow 0.0012685366
brightness 0.0043312507
elastic_transform 0.0055

In [11]:
import csv

with open('softmax_sorted2.csv', 'w', newline='') as csvfile:
    filewriter = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
    for key in augs_sorted.keys():
        filewriter.writerow([str(key)])
        for key2 in augs_sorted[key].keys():
            # print(key2, augs_sorted[key][key2])
            filewriter.writerow([str(key2), str(augs_sorted[key][key2])])

In [ ]:
import pickle
with open('/storage/users/rkashefi/rojina/new_contrastive/clip_vec/softmax_sorted2.pkl', 'rb') as file:
    loaded_data = pickle.load(file)

rearranged_data = {key: {inner_key: value for inner_key, value in inner_dict.items()} for key, inner_dict in loaded_data.items()}
df = pd.DataFrame.from_dict(rearranged_data, orient='index')
df = df.transpose()
df.columns = [
    "Airplane",
    "Automobile",
    "Bird",
    "Cat",
    "Deer",
    "Dog",
    "Frog",
    "Horse",
    "Ship",
    "Truck"
]
df.to_csv('result2.csv')

# Softmax (All images)

In [48]:
import pickle

In [ ]:
diffs = []

In [50]:
with open(f'./tensors/diffs_{args.aug}.pkl', 'wb') as f:
    pickle.dump(diffs, f)

In [ ]:
import os
import torch
augs = os.listdir('./outputs')
for aug in augs:
    with open(f'./outputs/{aug}', 'r') as file:
        lines = file.readlines()
        lines.pop(0)
    vals = []
    for line in lines:
        vals.append(float(line.split(':')[-1].split('\n')[0]))
    
    vals = torch.tensor(vals)
    vals_softmax = torch.nn.functional.softmax(vals).detach().cpu().numpy()
    with open(f'./softmax2/{aug}', 'w') as file:
        for val in vals_softmax:
            file.write(str(val)+'\n')

In [26]:
import os
import torch
augs = os.listdir('./outputs')
for aug in augs:
    with open(f'./outputs/{aug}', 'r') as file:
        lines = file.readlines()
        lines.pop(0)
    vals = []
    for line in lines:
        vals.append(float(line.split(':')[-1].split('\n')[0]))
    
    vals = torch.tensor(vals)
    vals_softmax = torch.nn.functional.softmax(vals).detach().cpu().numpy()
    with open(f'./softmax/{aug}', 'w') as file:
        for val in vals_softmax:
            file.write(str(val)+'\n')

/tmp/ipykernel_10848/2339142470.py:13: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  vals_softmax = torch.nn.functional.softmax(vals).detach().cpu().numpy()


In [48]:
augs = os.listdir('./softmax')
augs_probs = {}
for i in range(10):
    augs_probs[i] = {}

for aug in augs:
    with open(f'./softmax/{aug}', 'r') as file:
        lines = file.readlines()

    for i, line in enumerate(lines):
        # augs_probs[i].append({
        #     aug.split('.')[0] : float(line.split('\n')[0])
        # })
        augs_probs[i][aug.split('.')[0]] = float(line.split('\n')[0])


In [49]:
augs_probs[0]

{'random_crop': 0.0040205405,
 'rot90': 5.038615e-06,
 'gaussian_blur': 0.13409017,
 'gaussian_noise': 0.03901328,
 'motion_blur': 0.03116428,
 'jpeg_compression': 0.090828806,
 'shot_noise': 0.13746814,
 'flip': 0.11583374,
 'brightness': 0.11194416,
 'snow': 0.3783712,
 'fog': 0.4014634,
 'glass_blur': 1.2054588e-08,
 'elastic_transform': 0.06608733,
 'contrast': 0.11341642,
 'speckle_noise': 0.13079225,
 'color_jitter': 0.031809445,
 'saturate': 0.05244525,
 'zoom_blur': 0.04244273,
 'spatter': 0.5304918,
 'rot270': 3.2738772e-06,
 'pixelate': 0.030844223,
 'impulse_noise': 0.0028079278,
 'defocus_blur': 0.13135473}

In [51]:
augs_sorted = {}
for key in augs_probs.keys():
    augs_sorted[key] = {k: v for k, v in sorted(augs_probs[key].items(), key=lambda item: item[1])}

In [57]:
import pickle
with open('./softmax_sorted.pkl', 'wb') as file:
    pickle.dump(augs_sorted, file)

In [59]:
import pickle
with open('./softmax_sorted.pkl', 'rb') as file:
    oo = pickle.load(file)

In [62]:
for key in augs_sorted.keys():
    for key2 in augs_sorted[key].keys():
        print(key2, augs_sorted[key][key2])

glass_blur 1.2054588e-08
rot270 3.2738772e-06
rot90 5.038615e-06
impulse_noise 0.0028079278
random_crop 0.0040205405
pixelate 0.030844223
motion_blur 0.03116428
color_jitter 0.031809445
gaussian_noise 0.03901328
zoom_blur 0.04244273
saturate 0.05244525
elastic_transform 0.06608733
jpeg_compression 0.090828806
brightness 0.11194416
contrast 0.11341642
flip 0.11583374
speckle_noise 0.13079225
defocus_blur 0.13135473
gaussian_blur 0.13409017
shot_noise 0.13746814
snow 0.3783712
fog 0.4014634
spatter 0.5304918
speckle_noise 0.000108593915
shot_noise 0.00057963433
color_jitter 0.0011954387
spatter 0.002020797
impulse_noise 0.002637804
snow 0.010860088
saturate 0.011297863
gaussian_noise 0.01842857
random_crop 0.027475407
pixelate 0.034814823
jpeg_compression 0.04158446
fog 0.048989244
flip 0.08968437
gaussian_blur 0.09388468
defocus_blur 0.09428844
motion_blur 0.09942751
brightness 0.10103441
contrast 0.10612949
zoom_blur 0.1277044
elastic_transform 0.14662114
rot90 0.30168277
rot270 0.4348

In [63]:
import csv

with open('softmax_sorted.csv', 'w', newline='') as csvfile:
    filewriter = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
    for key in augs_sorted.keys():
        filewriter.writerow([str(key)])
        for key2 in augs_sorted[key].keys():
            # print(key2, augs_sorted[key][key2])
            filewriter.writerow([str(key2), str(augs_sorted[key][key2])])

In [2]:
import pickle
import pandas as pd
with open('/storage/users/rkashefi/rojina/new_contrastive/clip_vec/softmax_sorted.pkl', 'rb') as file:
    loaded_data = pickle.load(file)

rearranged_data = {key: {inner_key: value for inner_key, value in inner_dict.items()} for key, inner_dict in loaded_data.items()}
df = pd.DataFrame.from_dict(rearranged_data, orient='index')
df = df.transpose()
df.columns = [
    "Airplane",
    "Automobile",
    "Bird",
    "Cat",
    "Deer",
    "Dog",
    "Frog",
    "Horse",
    "Ship",
    "Truck"
]
df.to_csv('result.csv')

# Compare three methods

In [1]:
import pandas as pd

In [2]:
text = pd.read_csv('./result3.csv')
text.rename(columns={'Unnamed: 0': 'aug'}, inplace=True)
text_image = pd.read_csv('./result2.csv')
text_image.rename(columns={'Unnamed: 0': 'aug'}, inplace=True)
image = pd.read_csv('./result.csv')
image.rename(columns={'Unnamed: 0': 'aug'}, inplace=True)

In [6]:
import pandas as pd
def comparing(df):
    for column in df.columns:
        sorted_df = df.sort_values(by=column, ascending=False)
        corresponding_values_B = sorted_df['aug'].tolist()
        print(f'{column}: {corresponding_values_B}')

In [18]:
comparing(image)

aug: ['zoom_blur', 'speckle_noise', 'spatter', 'snow', 'shot_noise', 'saturate', 'rot90', 'rot270', 'random_crop', 'pixelate', 'motion_blur', 'jpeg_compression', 'impulse_noise', 'glass_blur', 'gaussian_noise', 'gaussian_blur', 'fog', 'flip', 'elastic_transform', 'defocus_blur', 'contrast', 'color_jitter', 'brightness']
Airplane: ['spatter', 'fog', 'snow', 'shot_noise', 'gaussian_blur', 'defocus_blur', 'speckle_noise', 'flip', 'contrast', 'brightness', 'jpeg_compression', 'elastic_transform', 'saturate', 'zoom_blur', 'gaussian_noise', 'color_jitter', 'motion_blur', 'pixelate', 'random_crop', 'impulse_noise', 'rot90', 'rot270', 'glass_blur']
Automobile: ['glass_blur', 'rot270', 'rot90', 'elastic_transform', 'zoom_blur', 'contrast', 'brightness', 'motion_blur', 'defocus_blur', 'gaussian_blur', 'flip', 'fog', 'jpeg_compression', 'pixelate', 'random_crop', 'gaussian_noise', 'saturate', 'snow', 'impulse_noise', 'spatter', 'color_jitter', 'shot_noise', 'speckle_noise']
Bird: ['snow', 'jpeg_c

In [19]:
comparing(text_image)

aug: ['zoom_blur', 'speckle_noise', 'spatter', 'snow', 'shot_noise', 'saturate', 'rot90', 'rot270', 'random_crop', 'pixelate', 'motion_blur', 'jpeg_compression', 'impulse_noise', 'glass_blur', 'gaussian_noise', 'gaussian_blur', 'fog', 'flip', 'elastic_transform', 'defocus_blur', 'contrast', 'color_jitter', 'brightness']
Airplane: ['jpeg_compression', 'flip', 'spatter', 'gaussian_noise', 'elastic_transform', 'snow', 'contrast', 'brightness', 'impulse_noise', 'speckle_noise', 'zoom_blur', 'shot_noise', 'saturate', 'random_crop', 'pixelate', 'glass_blur', 'color_jitter', 'fog', 'gaussian_blur', 'motion_blur', 'rot90', 'defocus_blur', 'rot270']
Automobile: ['pixelate', 'flip', 'spatter', 'jpeg_compression', 'elastic_transform', 'brightness', 'snow', 'gaussian_noise', 'zoom_blur', 'saturate', 'speckle_noise', 'impulse_noise', 'contrast', 'shot_noise', 'color_jitter', 'motion_blur', 'gaussian_blur', 'glass_blur', 'defocus_blur', 'random_crop', 'fog', 'rot90', 'rot270']
Bird: ['random_crop', 

In [20]:
comparing(text)

aug: ['zoom_blur', 'speckle_noise', 'spatter', 'snow', 'shot_noise', 'saturate', 'rot90', 'random_crop', 'pixelate', 'motion_blur', 'jpeg_compression', 'impulse_noise', 'gaussian_noise', 'gaussian_blur', 'fog', 'flip', 'elastic_transform', 'defocus_blur', 'contrast', 'color_jitter', 'brightness']
Airplane: ['jpeg_compression', 'contrast', 'random_crop', 'flip', 'snow', 'fog', 'color_jitter', 'pixelate', 'gaussian_noise', 'elastic_transform', 'spatter', 'brightness', 'motion_blur', 'impulse_noise', 'saturate', 'zoom_blur', 'gaussian_blur', 'shot_noise', 'defocus_blur', 'speckle_noise', 'rot90']
Automobile: ['brightness', 'snow', 'flip', 'zoom_blur', 'motion_blur', 'jpeg_compression', 'elastic_transform', 'pixelate', 'contrast', 'color_jitter', 'defocus_blur', 'saturate', 'impulse_noise', 'random_crop', 'gaussian_noise', 'fog', 'gaussian_blur', 'shot_noise', 'speckle_noise', 'spatter', 'rot90']
Bird: ['flip', 'snow', 'motion_blur', 'pixelate', 'contrast', 'fog', 'gaussian_noise', 'elasti